# Understanding retries
Here description of what this does.

In [1]:
import datetime
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import scan

import numpy as np
import pandas as pd

es = Elasticsearch([{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)


### select period

In [2]:
start_date = datetime.datetime(2017, 8, 10)
curre_date = datetime.datetime(2017, 8, 20, 23, 59, 59)
#curre_date = datetime.datetime(2017, 8, 28, 23, 59, 59)
#curre_date = datetime.datetime.utcnow()
days_of_data = (curre_date - start_date).days

### selecting indices to scan

In [3]:
# get job archive indices
indices = es.cat.indices(index="jobs_archive_20*", h="index", request_timeout=600).split('\n')
indices = sorted(indices)
indices = [x for x in indices if x != '']
#for i in indices:
#    print(i)

selected_indices=[] 
for d in range(days_of_data+1):
    td = start_date + datetime.timedelta(d)
    ind = 'jobs_archive_'+td.strftime('%Y-%m-%d')
    if ind in indices:
        selected_indices.append(ind)

job_indices = ','.join(selected_indices)
print(job_indices)

jobs_archive_2017-08-10,jobs_archive_2017-08-11,jobs_archive_2017-08-12,jobs_archive_2017-08-13,jobs_archive_2017-08-14,jobs_archive_2017-08-15,jobs_archive_2017-08-16,jobs_archive_2017-08-17,jobs_archive_2017-08-18,jobs_archive_2017-08-19,jobs_archive_2017-08-20


### running the scan

In [4]:
job_query = {
    "size": 0,
    "_source": [ "corecount", "exeerrorcode", "pandaid", "failedattempt", "child_ids", "wall_time", "jobstatus", "processingtype" ],
    'query':{
#        "match_all": {}
             'bool':{
                    'must':[                
                          { "term": {"pandaid": 3557118025 } },
#                         { "term": {"pandaid": 3556781963 } },
#                          { "term": {"pandaid": 3556868874 } },
#                         { "term": {"prodsourcelabel": "managed" } }
                     ],
#                    'should':[
                       # { "term": {"prodsourcelabel": "rc_test" } },
                       # { "term": {"prodsourcelabel": "prod_test" } },
                       # { "term": {"prodsourcelabel": "managed" } }
#                        { "term": {"jobstatus": "finished" } },
#                        { "term": {"jobstatus": "failed" } },
#                    ]
             }
    }
                
}

scroll = scan(client=es, index=job_indices, query=job_query, scroll='50m', timeout="50m", size=10000)

count = 0
for res in scroll:
    count += 1
    
    #print(res)
    #if count>10000: break
    
    r = res['_source']
    
    cores = r['corecount']
    errorcode = r['exeerrorcode']
    pandaid = r['pandaid']
    failedattempt = r['failedattempt']
    if 'child_ids' in r:
        child_ids = r['child_ids']
    else:
        child_ids=[]
    wall_time = r['wall_time']
    jobstatus = r['jobstatus']
    processingtype = r['processingtype']
    
    print(pandaid)
    print(child_ids)

3557118025
[]


### Debug

### Find initial jobs

### Make a jobgroup from starting from an initial job

### Print result